In [58]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
%matplotlib inline

# Load data



CRU TS4.01: Climatic Research Unit (CRU) Time-Series (TS) version 4.01 of high-resolution gridded data of month-by-month variation in climate (Jan. 1901- Dec. 2016)  

You can download CRU TS4.01 data set from CDEA Archive (needs registration)  
http://data.ceda.ac.uk/badc/cru/data/cru_ts/cru_ts_4.01/data/tmp  

Note that use of these data is covered by the following licence:  
http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/  


In [11]:
# read dat file
data = pd.read_csv("../data/cru_ts4.01.1901.2016.tmp.dat", delim_whitespace=True, header=None).values

In [30]:
# reshape
data = data.reshape((-1, 360*720))

In [31]:
# ignore invalid grids
vvv = (data == -999).sum(axis=0)

In [32]:
# same as the paper
(vvv == 0).sum()

67420

In [52]:
data_live = data[:, vvv == 0].transpose().astype("int32")

In [53]:
data_live.shape

(67420, 1392)

In [45]:
# seems 10 times degrees celsius
# -595 means -59.5 
# 395 means 39.5
data_live.max(), data_live.min()

(395, -595)

# Sample Data 1

We want similar situation as MODEL4 in the paper, that says 97% Accuracy.  
Here, we randomly pick grid and randomly pick year, around 500K in total. 

In [70]:
sample_size = 519718

np.random.seed(71)
sample_indices = np.random.choice(67420*77, sample_size, replace=False)
train_size = 519718 * 3 // 4

The definition of RISE/FALL in the paper is not clear for me, so simply compare 30 years mean / 10 years mean  
(the paper says based on the mean temperature for 10 years after the training period.)

In [68]:
X = np.zeros((sample_size, 360))
y = np.zeros(sample_size).astype("int")

for i in range(sample_size):
    sample_ind = sample_indices[i]
    grid_ind = sample_ind // 77
    year_ind = sample_ind % 77
    X[i, :] = data_live[grid_ind, (year_ind*12):(year_ind*12+360)]
    mean30 = X[i, :].mean()
    mean10 = data_live[grid_ind, (year_ind*12+360):(year_ind*12+480)].mean()
    if mean10 > mean30:
        y[i] = 1
    else:
        y[i] = 0

In [69]:
# around same distribution
(1-y).sum(), y.sum()

(156818, 362900)

# Build Model with LightGBM
### raw X

In [71]:
train_data = lgb.Dataset(X[:train_size, :], y[:train_size])
valid_data = lgb.Dataset(X[train_size:, :], y[train_size:])

In [78]:
params = {
    "objective" : "binary", 
    "metric" : "binary_error"
}

In [80]:
lgb.train(params, train_set=train_data, valid_sets=[valid_data], early_stopping_rounds=200, num_boost_round=10000, verbose_eval=100)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's binary_error: 0.222997
[200]	valid_0's binary_error: 0.194743
[300]	valid_0's binary_error: 0.176495
[400]	valid_0's binary_error: 0.160717
[500]	valid_0's binary_error: 0.149704
[600]	valid_0's binary_error: 0.140322
[700]	valid_0's binary_error: 0.133957
[800]	valid_0's binary_error: 0.126753
[900]	valid_0's binary_error: 0.121827
[1000]	valid_0's binary_error: 0.117663
[1100]	valid_0's binary_error: 0.113592
[1200]	valid_0's binary_error: 0.110167
[1300]	valid_0's binary_error: 0.106904
[1400]	valid_0's binary_error: 0.103817
[1500]	valid_0's binary_error: 0.10127
[1600]	valid_0's binary_error: 0.0989456
[1700]	valid_0's binary_error: 0.0967136
[1800]	valid_0's binary_error: 0.0944201
[1900]	valid_0's binary_error: 0.0925806
[2000]	valid_0's binary_error: 0.0911953
[2100]	valid_0's binary_error: 0.0895713
[2200]	valid_0's binary_error: 0.0882244
[2300]	valid_0's binary_error: 0.0871854
[2400]	valid_0's bi

In [87]:
1 - 0.0635573

0.9364427

### standardized X

In [83]:
X_st = X - X.min(axis=1, keepdims=True)
X_st = X_st / X_st.max(axis=1, keepdims=True)

In [84]:
train_data = lgb.Dataset(X_st[:train_size, :], y[:train_size])
valid_data = lgb.Dataset(X_st[train_size:, :], y[train_size:])

In [85]:
params = {
    "objective" : "binary", 
    "metric" : "binary_error"
}

In [86]:
lgb.train(params, train_set=train_data, valid_sets=[valid_data], early_stopping_rounds=200, num_boost_round=10000, verbose_eval=100)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's binary_error: 0.167321
[200]	valid_0's binary_error: 0.131663
[300]	valid_0's binary_error: 0.112376
[400]	valid_0's binary_error: 0.0990841
[500]	valid_0's binary_error: 0.0889633
[600]	valid_0's binary_error: 0.0815901
[700]	valid_0's binary_error: 0.0758639
[800]	valid_0's binary_error: 0.0709382
[900]	valid_0's binary_error: 0.06719
[1000]	valid_0's binary_error: 0.0637959
[1100]	valid_0's binary_error: 0.060725
[1200]	valid_0's binary_error: 0.0582544
[1300]	valid_0's binary_error: 0.0558532
[1400]	valid_0's binary_error: 0.0541138
[1500]	valid_0's binary_error: 0.0519895
[1600]	valid_0's binary_error: 0.0506657
[1700]	valid_0's binary_error: 0.0493189
[1800]	valid_0's binary_error: 0.0479797
[1900]	valid_0's binary_error: 0.0466559
[2000]	valid_0's binary_error: 0.0456861
[2100]	valid_0's binary_error: 0.0447395
[2200]	valid_0's binary_error: 0.0436851
[2300]	valid_0's binary_error: 0.043054
[2400]	val

In [88]:
1 - 0.0324559

0.9675441

# Sample Data 2

Now we sample train and valid data in time-series manner and check the Accuracy.  
For train data, we randomly pick grid and randomly pick year 0-54(start from 1901-1955), 375K in total.  
For valid data, we randomly pick grid and randomly pick year 64-76(start from 1965-1977), 375K in total.  
train data (both X and y) do not include 1995- data.  
valid_y consists of 1995- data.  

In [92]:
train_size = 375000

np.random.seed(71)
train_sample_indices = np.random.choice(67420*55, train_size, replace=False)

In [93]:
valid_size = 125000

np.random.seed(71)
valid_sample_indices = np.random.choice(67420*13, valid_size, replace=False)

In [94]:
train_X = np.zeros((train_size, 360))
train_y = np.zeros(train_size).astype("int")

for i in range(train_size):
    sample_ind = train_sample_indices[i]
    grid_ind = sample_ind // 55
    year_ind = sample_ind % 55
    train_X[i, :] = data_live[grid_ind, (year_ind*12):(year_ind*12+360)]
    mean30 = train_X[i, :].mean()
    mean10 = data_live[grid_ind, (year_ind*12+360):(year_ind*12+480)].mean()
    if mean10 > mean30:
        train_y[i] = 1
    else:
        train_y[i] = 0

In [96]:
valid_X = np.zeros((valid_size, 360))
valid_y = np.zeros(valid_size).astype("int")

for i in range(valid_size):
    sample_ind = valid_sample_indices[i]
    grid_ind = sample_ind // 13
    year_ind = sample_ind % 13 + 64
    valid_X[i, :] = data_live[grid_ind, (year_ind*12):(year_ind*12+360)]
    mean30 = valid_X[i, :].mean()
    mean10 = data_live[grid_ind, (year_ind*12+360):(year_ind*12+480)].mean()
    if mean10 > mean30:
        valid_y[i] = 1
    else:
        valid_y[i] = 0

In [97]:
(1-train_y).sum(), train_y.sum(), (1-valid_y).sum(), valid_y.sum()

(162195, 212805, 2613, 122387)

In [98]:
train_y.mean(), valid_y.mean()

(0.56748, 0.979096)

we see Global warming here...  
all 1 model has 98% accuracy.  

### raw X

In [103]:
train_data = lgb.Dataset(train_X, train_y)
valid_data = lgb.Dataset(valid_X, valid_y)

In [104]:
params = {
    "objective" : "binary", 
    "metric" : "binary_error"
}

In [105]:
lgb.train(params, train_set=train_data, valid_sets=[valid_data], early_stopping_rounds=200, num_boost_round=10000, verbose_eval=100)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's binary_error: 0.145256
[200]	valid_0's binary_error: 0.141512
[300]	valid_0's binary_error: 0.140848
[400]	valid_0's binary_error: 0.143792
Early stopping, best iteration is:
[288]	valid_0's binary_error: 0.139184


### standardized X

In [99]:
train_X_st = train_X - train_X.min(axis=1, keepdims=True)
train_X_st = train_X_st / train_X_st.max(axis=1, keepdims=True)
valid_X_st = valid_X - valid_X.min(axis=1, keepdims=True)
valid_X_st = valid_X_st / valid_X_st.max(axis=1, keepdims=True)

In [100]:
train_data = lgb.Dataset(train_X_st, train_y)
valid_data = lgb.Dataset(valid_X_st, valid_y)

In [101]:
params = {
    "objective" : "binary", 
    "metric" : "binary_error"
}

In [102]:
lgb.train(params, train_set=train_data, valid_sets=[valid_data], early_stopping_rounds=200, num_boost_round=10000, verbose_eval=100)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's binary_error: 0.112464
[200]	valid_0's binary_error: 0.114136
[300]	valid_0's binary_error: 0.117384
Early stopping, best iteration is:
[119]	valid_0's binary_error: 0.111232
